In [1]:
from datetime import datetime



StatementMeta(, e8cb938b-def3-4e73-8492-a76473c4391a, 3, Finished, Available, Finished)

In [2]:
today_date=datetime.today().strftime('%Y-%m-%d')


StatementMeta(, e8cb938b-def3-4e73-8492-a76473c4391a, 4, Finished, Available, Finished)

'2025-12-02'

In [2]:
abfs_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Files/Landing'
partition_path=f"/Processing_date={today_date}"
complete_path = abfs_path + partition_path
print(complete_path)

StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 4, Finished, Available, Finished)

abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Files/Landing/Processing_date=2025-12-02


In [3]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType, DateType

v_schema = StructType([
    StructField("Row_ID", StringType(), True),
    StructField("Order_ID", StringType(), True),
    StructField("Order_Date", DateType(), True),
    StructField("Ship_Date", DateType(), True),
    StructField("Ship_Mode", StringType(), True),
    StructField("Customer_ID", StringType(), True),
    StructField("Customer_Name", StringType(), True),
    StructField("Segment", StringType(), True),
    StructField("Postal_Code", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("Market", StringType(), True),
    StructField("Product_ID", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Sub_Category", StringType(), True),
    StructField("Product_Name", StringType(), True),
    StructField("Sales", DoubleType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Discount", DoubleType(), True),
    StructField("Profit", DoubleType(), True),
    StructField("Shipping_Cost", DoubleType(), True),
    StructField("Order_Priority", StringType(), True),
    StructField("Month", StringType(), True),
    StructField("Year", StringType(), True),
])



StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 5, Finished, Available, Finished)

In [4]:
df=spark.read.format('csv').option('header','True').schema(v_schema).load(complete_path)

StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 6, Finished, Available, Finished)

In [11]:
from pyspark.sql import functions as F

# Deduplicate source by merge keys
df_dedup = df.dropDuplicates(["Order_ID", "Customer_ID"])
df_dedup.createOrReplaceTempView('t_new_data')

t_new_data_columns = spark.sql("DESCRIBE t_new_data").select("col_name").rdd.flatMap(lambda x: x).collect()
print(t_new_data_columns)


StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 13, Finished, Available, Finished)

['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode', 'Customer_ID', 'Customer_Name', 'Segment', 'Postal_Code', 'City', 'State', 'Country', 'Region', 'Market', 'Product_ID', 'Category', 'Sub_Category', 'Product_Name', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost', 'Order_Priority', 'Month', 'Year']


In [7]:
%%sql
select * from t_new_data

StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 9, Finished, Available, Finished)

<Spark SQL result set with 848 rows and 4 fields>

In [8]:
Fabric_sales_bronze = 'abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/sales_bronze'

try:
    spark.read.format("delta").load(Fabric_sales_bronze)
except:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS sales_bronze(
        Row_ID STRING,
        Order_ID STRING,
        Order_Date DATE,
        Ship_Date DATE,
        Ship_Mode STRING,
        Customer_ID STRING,
        Customer_Name STRING,
        Segment STRING,
        Postal_Code STRING,
        City STRING,
        State STRING,
        Country STRING,
        Region STRING,
        Market STRING,
        Product_ID STRING,
        Category STRING,
        Sub_Category STRING,
        Product_Name STRING,
        Sales DOUBLE,
        Quantity INT,
        Discount DOUBLE,
        Profit DOUBLE,
        Shipping_Cost DOUBLE,
        Order_Priority STRING,
        Month STRING,
        Year STRING,
        Processing_date DATE
    )
    """)


StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 10, Finished, Available, Finished)

In [9]:
%%sql
select * from sales_bronze

StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 11, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 27 fields>

In [12]:
sql_statement= f"""MERGE INTO sales_bronze as target
                    USING t_new_data as source
                    on target.Order_ID = source.Order_ID and target.Customer_ID=source.Customer_ID

                    WHEN MATCHED THEN UPDATE SET 
                        target.Row_ID =source.Row_ID,
                        target.Order_ID =source.Order_ID,
                        target.Order_Date =source.Order_Date,
                        target.Ship_Date =source.Ship_Date,
                        target.Ship_Mode =source.Ship_Mode,
                        target.Customer_ID =source.Customer_ID,
                        target.Customer_Name =source.Customer_Name,
                        target.Segment =source.Segment,
                        target.Postal_Code =source.Postal_Code,
                        target.City =source.City,
                        target.State =source.State,
                        target.Country =source.Country,
                        target.Region =source.Region,
                        target.Market =source.Market,
                        target.Product_ID =source.Product_ID,
                        target.Category =source.Category,
                        target.Sub_Category =source.Sub_Category,
                        target.Product_Name =source.Product_Name,
                        target.Sales =source.Sales,
                        target.Quantity =source.Quantity,
                        target.Discount =source.Discount,
                        target.Profit =source.Profit,
                        target.Shipping_Cost =source.Shipping_Cost,
                        target.Order_Priority =source.Order_Priority,
                        target.Month =source.Month,
                        target.Year =source.Year,
                        target.Processing_date =DATE('{today_date}')

                    WHEN NOT MATCHED THEN INSERT (
                        Row_ID,
                        Order_ID,
                        Order_Date,
                        Ship_Date,
                        Ship_Mode,
                        Customer_ID,
                        Customer_Name,
                        Segment,                                
                        Postal_Code,
                        City,
                        State,
                        Country,
                        Region,
                        Market,
                        Product_ID,
                        Category,
                        Sub_Category,
                        Product_Name,
                        Sales,
                        Quantity,
                        Discount,
                        Profit,
                        Shipping_Cost,
                        Order_Priority,
                        Month,
                        Year,
                        Processing_date)
                        VALUES(
                            source.Row_ID,
                            source.Order_ID,
                            source.Order_Date,
                            source.Ship_Date,
                            source.Ship_Mode,
                            source.Customer_ID,
                            source.Customer_Name,
                            source.Segment,
                            source.Postal_Code,
                            source.City,
                            source.State,
                            source.Country,
                            source.Region,
                            source.Market,
                            source.Product_ID,
                            source.Category,
                            source.Sub_Category,
                            source.Product_Name,
                            source.Sales,
                            source.Quantity,
                            source.Discount,
                            source.Profit,
                            source.Shipping_Cost,
                            source.Order_Priority,
                            source.Month,
                            source.Year,
                            DATE('{today_date}')
                            )"""
spark.sql(sql_statement).show()


StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 14, Finished, Available, Finished)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|             1675|            1675|               0|                0|
+-----------------+----------------+----------------+-----------------+



In [13]:
%%sql
select * from sales_bronze

StatementMeta(, d93252df-2567-4f01-b3f0-8c048fed05d0, 15, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 27 fields>